In [ ]:
from pathlib import Path
import textwrap
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import xarray as xr
from typing import List, Tuple, Dict, Any

import seaborn as sns

strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

from sdm_eurec4a.visulization import (
    set_custom_rcParams,
    set_paper_rcParams,
    adjust_lightness_array,
    adjust_lightness,
    label_from_attrs,
    plot_one_one,
    add_additional_axis,
    add_subplotlabel,
    save_figure,
)
from sdm_eurec4a import RepositoryPath
from sdm_eurec4a import data_loading
from sdm_eurec4a.constants import TimeSlices
from sdm_eurec4a import conversions

from sdm_eurec4a.reductions import mean_and_stderror_of_mean


default_colors = set_paper_rcParams()
default_dark_colors = adjust_lightness_array(default_colors, 0.75)

RepoPaths = RepositoryPath("levante")

data_dir = RepoPaths.CLEO_data_dir / Path("output_v4.2")
data_dir_no_ventilation = RepoPaths.CLEO_data_dir / Path("output_v4.1")
data_dir_v40 = RepoPaths.CLEO_data_dir / Path("output_v4.0")

fig_dir = RepoPaths.fig_dir / Path("paper-v4.2")
fig_dir.mkdir(exist_ok=True, parents=False)

strength_cmap = sns.cubehelix_palette(start=0.5, rot=-0.5, as_cmap=True)

In [ ]:
from importlib import reload

reload(data_loading)
reload(conversions)

<module 'sdm_eurec4a.conversions' from '/home/m/m301096/repositories/sdm-eurec4a/src/sdm_eurec4a/conversions.py'>

# Data prepocessing

In [ ]:
ds_eulerian = xr.open_dataset(
    data_loading.__eulerian_data_path__(data_dir=data_dir, microphysic="coalbure_condensation_small")
)
ds_eulerian = ds_eulerian.sel(time=TimeSlices.full_state)
ds_eulerian = data_loading.__post_process_eulerian_dataset__(ds=ds_eulerian)

ds_conservation = xr.open_dataset(
    data_loading.__conservation_data_path__(data_dir=data_dir, microphysic="coalbure_condensation_small")
)
ds_conservation = ds_conservation.sel(time=TimeSlices.full_state)
ds_conservation = data_loading.__post_process_conservation_dataset__(
    ds=ds_conservation,
    da_surface_area=ds_eulerian["surface_area"].mean("gridbox"),
    timestep=ds_conservation["time"].diff("time").mean().values,
)

In [ ]:
microphysics_styles = data_loading.MicrophysicsStyles()

### Load the cleo output data

- Data in physical gridbox coordinates
- Data normalized by cloud base height.
- Data without ventilation coefficient

In [ ]:
cleo_dataset = data_loading.CleoDataset(
    data_dir=data_dir,
    microphysics=tuple(microphysics_styles),
)
# get physicsal height cleo output data
ds, ds_sem = cleo_dataset()
cleo_dataset.normalize_gridboxes()
# get normalized height cleo output data
ds_normalized, ds_normalized_sem = cleo_dataset()


# get non ventilated cleo output data
cleo_dataset_no_ventilation = data_loading.CleoDataset(
    data_dir=data_dir_no_ventilation,
    microphysics=tuple(microphysics_styles),
)
ds_no_ventilation, ds_sem_no_ventilation = cleo_dataset_no_ventilation()
# cleo_dataset.normalize_gridboxes()
# ds_normalized, ds_normalized_sem = cleo_dataset()

# valid_cleo_monitor_dataset = data_loading.CleoDataset(
#     data_dir= data_dir_v40,
#     microphysics=tuple(microphysics_styles),
# )
# # get physicsal height cleo output data
# ds_valid_cleo_monitor, ds_valid_cleo_monitor_sem = valid_cleo_monitor_dataset()

condensation
collision_condensation
coalbure_condensation_small
coalbure_condensation_large
condensation
collision_condensation
coalbure_condensation_small
coalbure_condensation_large


## Integrity of the combined dataset

there seems to be an issue, that the combined dataset can contain different values compared to the individual datasets.

Therefore, we identify the cloud ids, which have different values.
We omit these!

In [ ]:
atol = 1e-10
invalid_combined_dataset_ids = set()
error_combined_dataset_ids = set()

for mp in microphysics_styles:
    for cloud_id in ds["cloud_id"]:
        cloud_id = int(cloud_id.data)
        p = Path(
            f"/home/m/m301096/CLEO/data/output_v4.2/{mp}/cluster_{cloud_id}/processed/conservation_dataset.nc"
        )

        if p.is_file():
            ds_single = xr.open_dataset(p).sel(time=TimeSlices.quasi_stationary_state)
            inflow_diff = np.abs(
                ds_single["inflow"].mean("time").data
                - ds["inflow"].sel(microphysics=mp).sel(cloud_id=cloud_id).data
            )
            outflow_diff = np.abs(
                ds_single["outflow"].mean("time").data
                - ds["outflow"].sel(microphysics=mp).sel(cloud_id=cloud_id).data
            )
            source_diff = np.abs(
                ds_single["source"].mean("time").data
                - ds["source"].sel(microphysics=mp).sel(cloud_id=cloud_id).data
            )

            if inflow_diff > atol or outflow_diff > atol or source_diff > atol:
                invalid_combined_dataset_ids.add(cloud_id)

        else:
            error_combined_dataset_ids.add(cloud_id)

print(
    f"The following clouds have invalid data {invalid_combined_dataset_ids.union(error_combined_dataset_ids)}"
)

The following clouds have invalid data {159}


Invalid for none ventilation dataset 

In [ ]:
atol = 1e-10
invalid_combined_dataset_ids_no_ventilation = set()
error_combined_dataset_ids_no_ventilation = set()

for mp in microphysics_styles:
    for cloud_id in ds_no_ventilation["cloud_id"]:
        cloud_id = int(cloud_id.data)
        p = Path(
            f"/home/m/m301096/CLEO/data/output_v4.1/{mp}/cluster_{cloud_id}/processed/conservation_dataset.nc"
        )

        if p.is_file():
            ds_single = xr.open_dataset(p).sel(time=TimeSlices.quasi_stationary_state)
            inflow_diff = np.abs(
                ds_single["inflow"].mean("time").data
                - ds_no_ventilation["inflow"].sel(microphysics=mp).sel(cloud_id=cloud_id).data
            )
            outflow_diff = np.abs(
                ds_single["outflow"].mean("time").data
                - ds_no_ventilation["outflow"].sel(microphysics=mp).sel(cloud_id=cloud_id).data
            )
            source_diff = np.abs(
                ds_single["source"].mean("time").data
                - ds_no_ventilation["source"].sel(microphysics=mp).sel(cloud_id=cloud_id).data
            )

            if inflow_diff > atol or outflow_diff > atol or source_diff > atol:
                invalid_combined_dataset_ids_no_ventilation.add(cloud_id)

        else:
            error_combined_dataset_ids_no_ventilation.add(cloud_id)

print(
    f"The following clouds have invalid data {invalid_combined_dataset_ids_no_ventilation.union(error_combined_dataset_ids_no_ventilation)}"
)

The following clouds have invalid data {130}


Option to get more information about these clouds.More detailed analysis of the clouds with invalid data from concatenation or missing simulations

In [ ]:
# for cloud_id in invalid_combined_dataset_ids:
#     print('-------------')
#     print(cloud_id)


#     print(
#         'MP'.ljust(28),
#         'I-conc'.ljust(10),
#         'I-true'.ljust(10),
#         'I-DIFF'.ljust(10),
#         '|',
#         'O-conc'.ljust(10),
#         'O-true'.ljust(10),
#         'O-DIFF'.ljust(10),
#         '|',
#         'S-conc'.ljust(10),
#         'S-true'.ljust(10),
#         'S-DIFF'.ljust(10),
#     )

#     for mp in microphysics_styles :
#         p = Path(f'/home/m/m301096/CLEO/data/output_v4.2/{mp}/cluster_{cloud_id}/processed/conservation_dataset.nc')

#         if p.is_file():
#             ds_single = xr.open_dataset(p).sel(time = TimeSlices.quasi_stationary_state)

#             print(
#                 str(mp).ljust(28),
#                 f'{ds['inflow'].sel(microphysics = mp).sel(cloud_id = cloud_id ).data:.2e}'.ljust(10),
#                 f'{ds_single['inflow'].mean('time').data:.2e}'.ljust(10),
#                 f'{ds_single['inflow'].mean('time').data - ds['inflow'].sel(microphysics = mp).sel(cloud_id = cloud_id ).data:.2e}'.ljust(10),
#                 '|',
#                 f'{ds['outflow'].sel(microphysics = mp).sel(cloud_id = cloud_id).data:.2e}'.ljust(10),
#                 f'{ds_single['outflow'].mean('time').data:.2e}'.ljust(10),
#                 f'{ds_single['outflow'].mean('time').data - ds['outflow'].sel(microphysics = mp).sel(cloud_id = cloud_id ).data:.2e}'.ljust(10),
#                 '|',
#                 f'{ds['source'].sel(microphysics = mp).sel(cloud_id = cloud_id ).data:.2e}'.ljust(10),
#                 f'{ds_single['source'].mean('time').data:.2e}'.ljust(10),
#                 f'{ds_single['source'].mean('time').data - ds['source'].sel(microphysics = mp).sel(cloud_id = cloud_id ).data:.2e}'.ljust(10),
#                 # f'{ds_eulerian['massdelta_condensation'].sel(cloud_id = cloud_id).sum('gridbox').mean('time').data:.2e}'.ljust(10),
#                 )
#         else :
#             pass

## Integrity of conservation datasets

We have seen, that the conservation dataset seem to show some errors when it comes to the total value of $A = I+O+S-\frac{dR}{dt}$.

We can analyse this in detail below and find, that this is the case for some clouds.
We state, that the error should not exceed 10% compared to any of the conservation variables
- inflow
- outflow
- source

So it needs to be $A/V < 10 \% \,\, \forall \,\, V $ in $\set{I,O,S}$

#### Compare all datasets visually

Use only the temporal mean from the CLEO output dataset itself.

In [ ]:
# fig, axs = plt.subplots(nrows = 4, figsize = (8,6))


# for plot_dataset, label in zip(
#     [ds_valid_cleo_monitor, ds_no_ventilation, ds],
#     ['good monitor', 'non ventilated', 'ventilated'],
# ):
#     total = (
#         plot_dataset["inflow"] +
#         plot_dataset["outflow"] +
#         plot_dataset["source"] +
#         - plot_dataset["reservoir_change"]
#     )

#     error = {}
#     for key in ['inflow', 'outflow', 'source'] :
#         error[key] = total / plot_dataset[key] * 100
#         error[key].attrs.update(plot_dataset[key].attrs)
#         error[key].attrs.update(units = r'\%', description = f"Relative error of {key} per gridbox per cloud")
#         error[key] = error[key].expand_dims(which = [key])

#     da_error = xr.concat(
#         error.values(),
#         dim = 'which',
#     )
#     da_maximum_error = np.abs(da_error).max(dim = 'which').expand_dims(which = ['maximum'])
#     error['maximum'] = da_maximum_error

#     da_error = xr.concat(
#         error.values(),
#         dim = 'which',
#     )


#     for _ax, key in zip(axs, da_error['which'].data):
#         _ax.plot(
#             da_error.sel(which = key)['cloud_id'].astype(str),
#             np.abs(da_error.sel(which = key)).max('microphysics'),
#             label = label,
#             linestyle = '-',
#             marker = '+',
#         )
#         # _ax.plot(
#         #     da_error.sel(which = key)['cloud_id'].astype(str),
#         #     np.abs(da_error.sel(which = key)).sel(microphysics = 'condensation'),
#         #     label = label,
#         #     linestyle = ':',
#         #     marker = 'x',
#         # )
#         _ax.tick_params(axis='x', rotation=90)# .sel(which = 'maximum').max('microphysics')
#         _ax.set_title(key)
#         _ax.legend()
#         _ax.set_ylim(0, 10)

# # fig, axs = plt.subplots(
# #     nrows = 4,
# #     ncols = 1,
# #     sharey=True,
# #     figsize=(10, 10))

# # for _ax, key in zip(axs, error):
# #     data = error[key]
# #     pcm = _ax.pcolormesh(
# #         data['cloud_id'].astype(str),
# #         [microphysics_styles[mp]['name'] for mp in data['microphysics'].data],
# #         data,
# #         cmap = strength_cmap,
# #     )
# #     fig.colorbar(pcm , ax = _ax, label = label_from_attrs(data, linebreak=True))

# # for _ax in axs :
# #     _ax.set_xticks([])
# fig.tight_layout()

Calculate the error for all timesteps and AFTERWARDS calculate the temporal mean.

In [ ]:
# fig, axs = plt.subplots(nrows = 4, figsize = (8, 6))


# for plot_dataset, label, conservation_data_dir in zip(
#     [ds_valid_cleo_monitor, ds_no_ventilation, ds],
#     ['good monitor', 'non ventilated', 'ventilated'],
#     [data_dir_v40, data_dir_no_ventilation, data_dir],
# ):
#     conservation_list = []
#     for mp in microphysics_styles :
#         _ds = xr.open_dataset(
#             data_loading.__conservation_data_path__(data_dir=conservation_data_dir, microphysic=mp)
#         )
#         conservation_list.append(_ds.expand_dims(microphysics = [mp]))

#     select_ds_conservation = xr.concat(
#         conservation_list,
#         dim = 'microphysics',
#     )

#     total = (
#         select_ds_conservation["inflow"] +
#         select_ds_conservation["outflow"] +
#         select_ds_conservation["source"]
#         - select_ds_conservation["reservoir_change"]
#     )
#     total = total.sel(time = TimeSlices.quasi_stationary_state).mean('time')

#     error = {}
#     for key in ['inflow', 'outflow', 'source'] :
#         error[key] = total / plot_dataset[key] * 100
#         error[key].attrs.update(plot_dataset[key].attrs)
#         error[key].attrs.update(units = r'\%', description = f"Relative error of {key} per gridbox per cloud")
#         error[key] = error[key].expand_dims(which = [key])

#     da_error = xr.concat(
#         error.values(),
#         dim = 'which',
#     )
#     da_maximum_error = np.abs(da_error).max(dim = 'which').expand_dims(which = ['maximum'])
#     error['maximum'] = da_maximum_error

#     da_error = xr.concat(
#         error.values(),
#         dim = 'which',
#     )

#     for _ax, key in zip(axs, da_error['which'].data):
#         _ax.plot(
#             da_error.sel(which = key)['cloud_id'].astype(str),
#             np.abs(da_error.sel(which = key)).max('microphysics'),
#             label = label,
#             linestyle = '-',
#             marker = '+',
#         )
#         # _ax.plot(
#         #     da_error.sel(which = key)['cloud_id'].astype(str),
#         #     np.abs(da_error.sel(which = key)).sel(microphysics = 'condensation'),
#         #     label = label,
#         #     linestyle = ':',
#         #     marker = 'x',
#         # )
#         _ax.tick_params(axis='x', rotation=90)# .sel(which = 'maximum').max('microphysics')
#         _ax.set_title(key)
#         _ax.set_ylim(0, 10)
#         _ax.legend()

# # fig, axs = plt.subplots(
# #     nrows = 4,
# #     ncols = 1,
# #     sharey=True,
# #     figsize=(10, 10))

# # for _ax, key in zip(axs, error):
# #     data = error[key]
# #     pcm = _ax.pcolormesh(
# #         data['cloud_id'].astype(str),
# #         [microphysics_styles[mp]['name'] for mp in data['microphysics'].data],
# #         data,
# #         cmap = strength_cmap,
# #     )
# #     fig.colorbar(pcm , ax = _ax, label = label_from_attrs(data, linebreak=True))

# # for _ax in axs :
# #     _ax.set_xticks([])
# fig.tight_layout()

#### Select only the ventilation cloud_ids

In [ ]:
plot_dataset = ds
conservation_data_dir = data_dir
relative_to_variables = ["inflow", "outflow", "source"]

conservation_list = []
for mp in microphysics_styles:
    _ds = xr.open_dataset(
        data_loading.__conservation_data_path__(data_dir=conservation_data_dir, microphysic=mp)
    )
    conservation_list.append(_ds.expand_dims(microphysics=[mp]))

select_ds_conservation = xr.concat(
    conservation_list,
    dim="microphysics",
)

total = (
    select_ds_conservation["inflow"]
    + select_ds_conservation["outflow"]
    + select_ds_conservation["source"]
    - select_ds_conservation["reservoir_change"]
)
total = total.sel(time=TimeSlices.quasi_stationary_state).mean("time")

error = {}
for key in ["inflow", "outflow", "source"]:
    error[key] = total / plot_dataset[key] * 100
    error[key].attrs.update(plot_dataset[key].attrs)
    error[key].attrs.update(units=r"\%", description=f"Relative error of {key} per gridbox per cloud")
    error[key] = error[key].expand_dims(which=[key])

da_error = xr.concat(
    error.values(),
    dim="which",
)
da_maximum_error = (
    np.abs(da_error).sel(which=relative_to_variables).max(dim="which").expand_dims(which=["maximum"])
)
error["maximum"] = da_maximum_error

da_error = xr.concat(
    error.values(),
    dim="which",
)

# where is the error of the conversation larger than 10% relative to any of the inflow, outflow, source
invalid_derivate_mass_conservation_ids = set(
    da_error["cloud_id"]
    .where(da_error.sel(which="maximum").max("microphysics") >= 10, drop=True)
    .data.astype(int)
    .tolist()
)

print(f"The following clouds have invalid conservation of mass {invalid_derivate_mass_conservation_ids}")

The following clouds have invalid conservation of mass {549, 239, 83, 86, 88, 569, 250}


# Remove outliers

We omit the following clouds:
- Cloud base precipitation above the set value (see in code)
- Where the combined data is not the same as the individual datasets
- Where the conservation dataset has a relative error above 10%
- Where the evaporation exceeds 2 mm/h

## Maximum cloud base precipitation

We exclude clouds with precipitation which exceed the inter-cloud mean by more than 4 standard deviations. 

In [ ]:
data = ds["inflow_precipitation"].sel(microphysics="condensation")

m, s = data.mean("cloud_id").data, data.std("cloud_id").data
print(f"mean: {m:.2f}, std: {s:.2f} mm/h")
print(f"mean + 4 std: {m + 4 * s:.2f} mm/h")
invalid_cloud_base_precipitation_ids = set(
    [int(_d) for _d in ds["cloud_id"].where(data > m + 4 * s, drop=True).data]
)
invalid_cloud_base_precipitation_ids

mean: 1.74, std: 5.35 mm/h
mean + 4 std: 23.15 mm/h


{384}

## Maximum column integrated evaporation

We exclude clouds with column integrated evaporation, which exceed the inter-cloud mean by more than 4 standard deviations.

In [ ]:
data = -ds["source_precipitation"].sel(microphysics="condensation")
m, s = data.mean("cloud_id").data, data.std("cloud_id").data
print(f"mean: {m:.2f}, std: {s:.2f} mm/h")
print(f"mean + 4 std: {m + 4 * s:.2f} mm/h")
invalid_column_integrated_evaporation_ids = set(
    [int(_d) for _d in ds["cloud_id"].where(data > m + 4 * s, drop=True).data]
)
invalid_column_integrated_evaporation_ids

mean: 0.26, std: 0.51 mm/h
mean + 4 std: 2.30 mm/h


{150, 384}

In [ ]:
all_cloud_ids = set(ds["cloud_id"].data.astype(int).tolist())

# remove clouds with invalid data
set_valid_cloud_ids = (
    all_cloud_ids
    - invalid_combined_dataset_ids.union(error_combined_dataset_ids)
    - invalid_derivate_mass_conservation_ids
)

# remove clouds with invalid precipitation and evaporation
set_valid_cloud_ids = (
    set_valid_cloud_ids
    - invalid_cloud_base_precipitation_ids
    - invalid_column_integrated_evaporation_ids
)

valid_cloud_ids = sorted(set_valid_cloud_ids)
print(f"Number of cloud with valid CLEO data is {len(valid_cloud_ids)} of {len(all_cloud_ids)}")

Number of cloud with valid CLEO data is 116 of 126


In [ ]:
ds_eulerian = ds_eulerian.sel(cloud_id=valid_cloud_ids)
ds_conservation = ds_conservation.sel(cloud_id=valid_cloud_ids)

ds = ds.sel(cloud_id=valid_cloud_ids)
ds_sem = ds_sem.sel(cloud_id=valid_cloud_ids)

ds_normalized = ds_normalized.sel(cloud_id=valid_cloud_ids)
ds_normalized_sem = ds_normalized_sem.sel(cloud_id=valid_cloud_ids)

ds_no_ventilation = ds_no_ventilation.sel(cloud_id=valid_cloud_ids)
ds_sem_no_ventilation = ds_sem_no_ventilation.sel(cloud_id=valid_cloud_ids)

# Data and Methods



### Precipitation plot

We want to show the stationary state of the simulation.
And we want to show the values of precipitation to show the errors.

In [ ]:
rolling_indices = 30
xlim = (0, 3600)
ylim = (0, 20)

fig, axs = plt.subplots(ncols=2, width_ratios=[1, 0.5])
ax: plt.Axes = axs[0]
ax_hist: plt.Axes = axs[1]
# ax_hist.sharey(ax)

x = ds_conservation["time"]

y = -ds_conservation["outflow_precipitation"].transpose("time", ...)
y_rolling = y.rolling(time=rolling_indices, center=True).mean()
x_rolling = x.rolling(time=rolling_indices, center=True).mean()

y_mean, y_sem = mean_and_stderror_of_mean(y.sel(time=TimeSlices.quasi_stationary_state), dims=("time",))

N = y_mean.sizes["cloud_id"]
# Inter-model spread (std of model means)
inter_model_spread = y_mean.std(dim="cloud_id", ddof=1) / np.sqrt(N)
# Individual model uncertainty propagation
individual_model_error = np.sqrt((y_sem**2).sum("cloud_id") / N)
# Total propagated SEM
total_sem = np.sqrt(inter_model_spread**2 + individual_model_error**2)
total_mean = y_mean.mean("cloud_id")

x = x.isel(time=slice(0, -2))
y = y.isel(time=slice(0, -2))

# ax.plot(
#     x,
#     y,
#     color = default_colors[0],
#     alpha = 0.2,
# );
ax.plot(
    x_rolling,
    y_rolling,
    color="grey",
    alpha=0.2,
)
ax.plot(
    x,
    y.mean("cloud_id"),
    color="k",
    alpha=1,
)

hist_bins = np.arange(0, 30, 0.5)
ax_hist.hist(
    y_mean,
    bins=hist_bins,
    orientation="horizontal",
    color="grey",
    alpha=0.2,
)

for _ax in [ax, ax_hist]:
    _ax.set_ylim(0, 30)
    _ax.axhline(
        total_mean.data,
        color="orange",
        linewidth=2,
        linestyle="-",
        label="Inter-Cloud-Mean:\n"
        + rf"{total_mean.data:.2f} $\pm$ {total_sem.data:.2f} {label_from_attrs(y, return_name=False)}",
    )
    _ax.fill_between(
        _ax.get_xlim(),
        total_mean - total_sem,
        total_mean + total_sem,
        color="orange",
        alpha=0.2,
    )

yticks = np.arange(min(ylim), max(ylim) + 1, 5)

ax.set_ylim(ylim)
ax.set_ylabel(label_from_attrs(y, linebreak=True))
ax.set_xlabel(label_from_attrs(x))
ax.set_xlim(xlim)
ax.set_yticks(yticks)
ax.legend(loc="upper center")

ax.axvline(1500, color="red", linestyle=":", alpha=0.5, zorder=1, label="Stationary State")

ax_hist.set_xlabel("Number of\nclouds")
ax_hist.set_ylim(ylim)
ax_hist.set_yticks(yticks, labels=np.full_like(yticks, "", dtype=str))

fig.tight_layout()

save_figure(fig=fig, filepath=fig_dir / f"precipitation-temporal-evolution")

# Results

## 1. EvapOnly setup analysis 

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1)

x = ds["inflow_precipitation"]
c = -ds["source_precipitation"]

y = ds["outflow"] + ds["source"] + ds["inflow"] - ds["reservoir_change"]
y = y / x * 100

y.attrs.update(
    units=r"\%",
    long_name=f"Rel. error of conservation against {x.attrs['long_name']}",
    description=f"Relative error of the sum of all conservation values against the variable {x.attrs['long_name']}",
)


for mp in ["condensation"]:
    sc = ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        c=c.sel(microphysics=mp),
        cmap=strength_cmap,
    )

fig.colorbar(sc, ax=ax, label=label_from_attrs(c, name_width=25))
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y, name_width=25))

Text(0, 0.5, 'Rel. error of\nconservation against\nCloud Base Precipitation\nFlux $\\left[  \\%  \\right]$')

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=2, figsize=(5, 5))

ax_x_hist: plt.Axes = axs[0]
ax_y_hist: plt.Axes = axs[1]

ax_y_hist.sharey(ax_x_hist)

x = -ds["source_precipitation"]
y = ds["evaporation_fraction"]

x_bins = np.arange(0, 1.3, 0.05)
y_bins = np.arange(0, 101, 5)

for mp in ["condensation"]:
    ax_x_hist.hist(
        x.sel(microphysics=mp),
        bins=x_bins,
        histtype="step",
        color=microphysics_styles.get_style(mp)["color"],
        lw=2,
    )
    ax_y_hist.hist(
        y.sel(microphysics=mp),
        bins=y_bins,
        histtype="step",
        color=microphysics_styles.get_style(mp)["color"],
        lw=2,
    )

ax_y_hist.set_xlabel(label_from_attrs(y))

x_ticks = xr.DataArray(ax_x_hist.get_xticks(), attrs=x.attrs.copy())

new_x_ticks = conversions.EvaporationUnits(data=x_ticks, input_type="precipitation").convert_to("energy")
new_ticks_func = lambda _: [f"{round(new_x, 0):.0f}" for x, new_x in zip(x_ticks, new_x_ticks.data)]
add_additional_axis(
    ax=ax_x_hist,
    new_ticks_func=new_ticks_func,
    label=label_from_attrs(da=new_x_ticks),
    position="bottom",
    offset_position=["axes", -0.4],
)
ax_x_hist.set_xlabel(label_from_attrs(da=x))


ax_x_hist.set_ylabel("Counts")
ax_y_hist.set_ylabel("Counts")


fig.tight_layout()

save_figure(fig=fig, filepath=fig_dir / f"{x.name}-{y.name}-histograms")

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(5, 5), width_ratios=[1, 0.3], height_ratios=[0.3, 1])

ax_empty = axs[0, 1]
ax_empty.axis("off")
ax_x_hist = axs[0, 0]
ax_y_hist = axs[1, 1]
ax_scatter = axs[1, 0]

ax_x_hist.sharex(ax_scatter)
ax_y_hist.sharey(ax_scatter)

x = -ds["source_precipitation"]
y = ds["evaporation_fraction"]
c = ds["cloud_liquid_water_content"]

x_bins = np.arange(0, 1.3, 0.05)
ax_scatter.set_xlim(x_bins[0], x_bins[-1])

y_bins = np.arange(0, 101, 5)
ax_scatter.set_ylim(y_bins[0], y_bins[-1])

for mp in ["condensation"]:
    ax_scatter.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        **microphysics_styles.get_style(mp),
    )
    ax_x_hist.hist(
        x.sel(microphysics=mp),
        bins=x_bins,
        histtype="step",
        color=microphysics_styles.get_style(mp)["color"],
        lw=2,
    )
    ax_y_hist.hist(
        y.sel(microphysics=mp),
        bins=y_bins,
        histtype="step",
        color=microphysics_styles.get_style(mp)["color"],
        lw=2,
        orientation="horizontal",
    )

ax_scatter.set_xlabel(label_from_attrs(x))
ax_scatter.set_ylabel(label_from_attrs(y))

ax_x_hist.set_ylabel("Counts")
ax_y_hist.set_xlabel("Counts")

x_ticks = xr.DataArray(ax_scatter.get_xticks(), attrs=x.attrs.copy())

new_x_ticks: xr.DataArray = conversions.EvaporationUnits(
    data=x_ticks, input_type="precipitation"
).convert_to("energy")
new_ticks_func = lambda _: [f"{round(new_x, 0):.0f}" for x, new_x in zip(x_ticks, new_x_ticks.data)]
add_additional_axis(
    ax=ax_scatter,
    new_ticks_func=new_ticks_func,
    label=label_from_attrs(da=new_x_ticks),
    position="bottom",
    offset_position=["axes", -0.3],
)
ax_scatter.set_xlabel(label_from_attrs(da=x))


for _ax in axs.flatten():
    _ax.grid(linestyle="-", alpha=0.2, color="grey")

fig.tight_layout()
save_figure(fig=fig, filepath=fig_dir / f"{x.name}-{y.name}-scattered-histograms-LINEAR")

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(5, 5), width_ratios=[1, 0.3], height_ratios=[0.3, 1])

ax_empty = axs[0, 1]
ax_empty.axis("off")
ax_x_hist = axs[0, 0]
ax_y_hist = axs[1, 1]
ax_scatter = axs[1, 0]

ax_x_hist.sharex(ax_scatter)
ax_y_hist.sharey(ax_scatter)

x = -ds["source_precipitation"]
y = ds["evaporation_fraction"]
c = ds["cloud_liquid_water_content"]

x_bins = np.geomspace(1e-3, 1.25, 30)
ax_scatter.set_xscale("log")
ax_scatter.set_xlim(x_bins[0], x_bins[-1])

y_bins = np.geomspace(1, 101, 30)
ax_scatter.set_ylim(y_bins[0], y_bins[-1])
ax_scatter.set_yscale("log")

for mp in ["condensation"]:
    ax_scatter.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        **microphysics_styles.get_style(mp),
    )
    ax_x_hist.hist(
        x.sel(microphysics=mp),
        bins=x_bins,
        histtype="step",
        color=microphysics_styles.get_style(mp)["color"],
        lw=2,
    )
    ax_y_hist.hist(
        y.sel(microphysics=mp),
        bins=y_bins,
        histtype="step",
        color=microphysics_styles.get_style(mp)["color"],
        lw=2,
        orientation="horizontal",
    )

ax_scatter.set_xlabel(label_from_attrs(x))
ax_scatter.set_ylabel(label_from_attrs(y))

ax_x_hist.set_ylabel("Counts")
ax_y_hist.set_xlabel("Counts")


for _ax in axs.flatten():
    _ax.grid(linestyle="-", alpha=0.2, color="grey")

fig.tight_layout()
save_figure(fig=fig, filepath=fig_dir / f"{x.name}-{y.name}-scattered-histograms-LOG")

## Evaporation Fraction

#### Create the theoretical ventilation lines 

In [ ]:
ds["radius_bins"].attrs = dict(
    long_name="Radius",
    units="$\\mu m$",
)
radii_label = label_from_attrs(ds["radius_bins"])

In [ ]:
RH = ds["relative_humidity"].mean() / 100
H = 1000

rhow = 0.998e3
rhoa = 1.2
eta = 1.85e-5
g = 9.81
nu = eta / rhoa
T = 294.41807507
p = 1e5
Dv0 = 0.211 * (T / 273.15) ** (1.94) * (1013.25e2 / p) * 1e-4  # PK97 (13-3)
Sc = 0.71  # nu/Dv0
gamma = 73e-3
Coo = 0.26
Cgamma = 18.4
lgamma = np.sqrt(gamma / (rhow * g))
kb = 1.380649e-23
Rconst = 8.314
Rv = 461.5
lv = 2.5e6
ka = 26.19e-3


def psat_water(T):
    theta = T - 273.15
    psat = 6.1121e2 * np.exp((18.678 - theta / 234.5) * (theta / (257.14 + theta)))
    return psat


def rhosat_water(T):
    rho = psat_water(T) * 18.01528e-3 / (Rconst * T)
    return rho


Dv = Dv0 / (1 + lv * Dv0 * rhosat_water(T) / (ka * T) * (lv / (Rv * T) - 1))


def theoretical_evaporation_fraction(r0s: xr.DataArray) -> xr.DataArray:
    bU = np.sqrt(8 / 3 * rhow / rhoa * g / 0.5)
    dr52 = 5 / 2 * Dv * H / bU * (1 - RH) * rhosat_water(T) / rhow
    efftheo = 1 - (1 - dr52 / r0s ** (5 / 2)) ** (6 / 5)
    efftheo = np.minimum(efftheo.fillna(1), 1)
    return efftheo


def fv(a, v):
    """Arguments are mass and velocity"""
    Re = 2 * a * np.abs(v) / nu
    x = Sc ** (1 / 3) * Re ** (1 / 2)
    if a < 60e-6:
        return 1 + 0.108 * x**2
    else:
        return 0.78 + 0.308 * x


def fv_xr(a: xr.DataArray, v: xr.DataArray) -> xr.DataArray:
    """Arguments are mass and velocity"""
    Re = 2 * a * np.abs(v) / nu
    x = Sc ** (1 / 3) * Re ** (1 / 2)
    low = 1 + 0.108 * x**2
    high = 0.78 + 0.308 * x

    return xr.where(a < 60e-6, low, high)


def vtlim(a):
    """Terminal velocity in m/s"""
    c1 = Coo ** (1 / 2)
    c2 = (12 * nu / a) ** (1 / 2)
    c3 = (8 * rhow * g * a / (3 * rhoa)) ** (1 / 2)
    return ((np.sqrt(c2**2 + 4 * c1 * c3) - c2) / (2 * c1)) ** 2


def vt(a):
    """Terminal velocity in m/s"""
    c1 = Coo ** (1 / 2) * (1 + Cgamma * (a / lgamma) ** 3) ** (1 / 6)
    c2 = (12 * nu / a) ** (1 / 2)
    c3 = (8 * rhow * g * a / (3 * rhoa)) ** (1 / 2)
    return ((np.sqrt(c2**2 + 4 * c1 * c3) - c2) / (2 * c1)) ** 2


ventilation_coefficient = fv_xr(ds["radius_bins"] * 1e-6, vt(ds["radius_bins"] * 1e-6))

fig, ax = plt.subplots(figsize=(4, 4))
ax.plot(ds["radius_bins"], ventilation_coefficient, label="fv", linestyle="-")
ax.set_xlabel(radii_label)
ax.set_ylabel("fv")
ax.set_ylim(0, 17)
# ax.legend()

plt.figure()
evaporation_fraction = theoretical_evaporation_fraction(ds["radius_bins"] * 1e-6)
evaporation_fraction_ventilation = evaporation_fraction * ventilation_coefficient
evaporation_fraction_ventilation: xr.DataArray = np.minimum(
    evaporation_fraction_ventilation.fillna(1), 1
)

plt.plot(
    ds["radius_bins"],
    evaporation_fraction,
    label="no ventilation",
)
plt.plot(
    ds["radius_bins"],
    evaporation_fraction_ventilation,
    label="ventilation",
)
plt.xlabel(radii_label)
plt.xlim(1e1, None)
plt.ylabel("Evaporation fraction")
plt.loglog()
plt.legend()

#### Compare no ventilation with ventilation 

In [ ]:
x = ds["evaporation_fraction"].sel(microphysics="condensation")
x_no_ventilation = ds_no_ventilation["evaporation_fraction"].sel(microphysics="condensation")


plt.hist(
    x_no_ventilation,
    bins=np.arange(0, 101, 5),
    label="No ventilation",
    alpha=1,
    linewidth=2,
    histtype="step",
)
plt.hist(x, bins=np.arange(0, 101, 5), label="With ventilation", alpha=1, linewidth=2, histtype="step")
plt.xlabel(label_from_attrs(x))
plt.ylabel("Counts")
plt.legend()

save_figure(fig=fig, filepath=fig_dir / f"ventilation-{x.name}-histograms")

In [ ]:
fig, ax = plt.subplots(1, 1)

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

# x_no_ventilation = ds_no_ventilation["cloud_mass_radius_mean"]
# y_no_ventilation = ds_no_ventilation["evaporation_fraction"]


for mp in ["condensation"]:
    style = microphysics_styles.get_style(mp)
    style["label"] += r" $\mathbf{with} \, f_v$"
    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        **style,
    )

# ax.plot(
#     x.transpose('microphysics', ...),
#     y.transpose('microphysics', ...),
#     color = 'black',
#     alpha = 0.1,
#     zorder = 1,
# )

ax.plot(
    ds["radius_bins"],
    1e2 * evaporation_fraction_ventilation,
    label=r"Theory $\mathbf{with} \, f_v$",
    color="black",
    linestyle="--",
)

ax.plot(
    ds["radius_bins"],
    1e2 * evaporation_fraction,
    label=r"Theory no $f_v$",
    color="grey",
    linestyle="--",
)

ax.set_xlim(50, None)
ax.set_ylim(1, None)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))
ax.legend(loc="lower left")
fig.tight_layout()

save_figure(fig=fig, filepath=fig_dir / f"theoretical-scatter-{x.name}-{y.name}")

In [ ]:
fig, ax = plt.subplots(1, 1)

x = ds["cloud_mass_radius_mean"]
y = ds["evaporation_fraction"]

# x_no_ventilation = ds_no_ventilation["cloud_mass_radius_mean"]
# y_no_ventilation = ds_no_ventilation["evaporation_fraction"]


for mp in microphysics_styles:
    style = microphysics_styles.get_style(mp)
    # style['label'] += r' $\mathbf{with} \, f_v$'
    style["label"] = None
    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        **style,
    )

ax.plot(
    x.transpose("microphysics", ...),
    y.transpose("microphysics", ...),
    color="black",
    alpha=0.1,
    zorder=1,
)

ax.plot(
    ds["radius_bins"],
    1e2 * evaporation_fraction_ventilation,
    label=r"Theory $\mathbf{with} \, f_v$",
    color="black",
    linestyle="--",
)

ax.plot(
    ds["radius_bins"],
    1e2 * evaporation_fraction,
    label=r"Theory no $f_v$",
    color="grey",
    linestyle="--",
)

ax.set_xlim(50, None)
ax.set_ylim(1, None)
ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y))
ax.legend(loc="lower left")
fig.tight_layout()

save_figure(fig=fig, filepath=fig_dir / f"theoretical-scatter-{x.name}-{y.name}-ALL")

In [ ]:
fig, ax = plt.subplots(1, 1)
# ax_no_ventilation = plt.Axes = axs[1]

x = ds["inflow_precipitation"]
y = -ds["source_precipitation"]

# x_no_ventilation = ds_no_ventilation["inflow_precipitation"]
# y_no_ventilation = - ds_no_ventilation["source_precipitation"]


for mp in ["condensation"]:
    style = microphysics_styles.get_style(mp)
    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        **style,
    )

ax.set_xscale("log")
ax.set_yscale("log")
# ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
lims = np.array([1e-6, 2.5e1])
ax.set_ylim(lims.min(), lims.max())
ax.set_xlim(1e-4, lims.max())

p_x_values = np.geomspace(lims.min(), lims.max(), 100)

values_label_size = 10

for p in [1, 0.1, 0.01]:
    style = dict(color="grey", alpha=p ** (1 / 5))
    lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
    line = lines[0]
    _x = 15
    _y = p * _x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(_x, _y),
        xytext=(10, 10),
        textcoords="offset points",
        va="top",
        ha="left",
        size=values_label_size,
        **style,
    )
    _x = 1e-3
    _y = p * _x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(_x, _y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=values_label_size,
        **style,
    )
    # _y = 2e-4
    # _x = _y / p
    # ax.annotate(
    #     f"{100 * p:.0f} %",
    #     xy=(_x, _y),
    #     xytext=(1, 1),
    #     textcoords="offset points",
    #     va="top",
    #     ha="left",
    #     size=values_label_size,
    #     **style,
    # )

ax.set_xlabel(label_from_attrs(ds["inflow_precipitation"]))
ax.set_ylabel(label_from_attrs(ds["source_precipitation"], name_width=20))

fig.tight_layout()

save_figure(fig=fig, filepath=fig_dir / f"{x.name}-{y.name}-scatter")

In [ ]:
fig, ax = plt.subplots(1, 1)
# ax_no_ventilation = plt.Axes = axs[1]

x = ds["inflow_precipitation"]
y = -ds["source_precipitation"]

# x_no_ventilation = ds_no_ventilation["inflow_precipitation"]
# y_no_ventilation = - ds_no_ventilation["source_precipitation"]


for mp in microphysics_styles:
    style = microphysics_styles.get_style(mp)
    ax.scatter(
        x.sel(microphysics=mp),
        y.sel(microphysics=mp),
        **style,
    )

# for _ax in axs:
#     _ax.set_xscale("log")
#     _ax.set_yscale("log")
#     _ax.set_xlabel(label_from_attrs(x))
#     _ax.set_ylabel(label_from_attrs(y))


ax.plot(
    x.transpose("microphysics", ...),
    y.transpose("microphysics", ...),
    color="grey",
    alpha=0.5,
)

ax.set_xscale("log")
ax.set_yscale("log")
# ax.set_yscale('symlog', linthresh = 1e-6, linscale = 0.2)
lims = np.array([1e-6, 2.5e1])
ax.set_ylim(lims.min(), lims.max())
ax.set_xlim(1e-4, lims.max())

p_x_values = np.geomspace(lims.min(), lims.max(), 100)

values_label_size = 10

for p in [1, 0.1, 0.01]:
    style = dict(color="grey", alpha=p ** (1 / 5))
    lines = ax.plot(p_x_values, p * p_x_values, "--", linewidth=1, zorder=0, **style)
    line = lines[0]
    _x = 15
    _y = p * _x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(_x, _y),
        xytext=(10, 10),
        textcoords="offset points",
        va="top",
        ha="left",
        size=values_label_size,
        **style,
    )
    _x = 1e-3
    _y = p * _x
    # y = 1e-0
    # x =(1/1.3) * (y / p)
    ax.annotate(
        f"{100 * p:.0f} %",
        xy=(_x, _y),
        xytext=(0, 0),
        textcoords="offset points",
        va="top",
        ha="left",
        size=values_label_size,
        **style,
    )
    # _y = 2e-4
    # _x = _y / p
    # ax.annotate(
    #     f"{100 * p:.0f} %",
    #     xy=(_x, _y),
    #     xytext=(1, 1),
    #     textcoords="offset points",
    #     va="top",
    #     ha="left",
    #     size=values_label_size,
    #     **style,
    # )

ax.set_xlabel(label_from_attrs(ds["inflow_precipitation"]))
ax.set_ylabel(label_from_attrs(ds["source_precipitation"], name_width=20))

fig.tight_layout()

save_figure(fig=fig, filepath=fig_dir / f"{x.name}-{y.name}-scatter-ALL")

## Evaporation Profiles

In [ ]:
def propagate_error(data, data_std, dim: str):

    N = len(data[dim])

    # Inter-model spread (std of model means)
    inter_model_spread = data.std(dim=dim, ddof=1) / np.sqrt(N)

    # Individual model uncertainty propagation
    individual_model_error = np.sqrt((data_std**2).sum(dim)) / N

    # Total propagated SEM
    total_sem = np.sqrt(inter_model_spread**2 + individual_model_error**2)

    return total_sem

In [ ]:
y_ticks = np.arange(0, 1.01, 0.25)

fig = plt.figure()
gs = fig.add_gridspec(nrows=1, ncols=1)

ax = fig.add_subplot(gs[:, :])

plot_microphysics = ["condensation"]

x = -ds_normalized["evaporation_rate_energy"]
x_sem = -ds_normalized_sem["evaporation_rate_energy"]
attrs = x.attrs.copy()
y = ds_normalized["normalized_gridbox_coord3"]

for mp in plot_microphysics:
    _x = x.sel(microphysics=mp)
    _x_sem = x_sem.sel(microphysics=mp)
    md_mean = _x.mean("cloud_id")
    md_sem = propagate_error(_x, _x_sem, dim="cloud_id")
    style_full = microphysics_styles[mp]

    ax.plot(
        _x.T,
        y.T,
        color=adjust_lightness(style_full["light_color"], 2),
        alpha=0.1,
        zorder=1,
    )

    ax.plot(
        md_mean,
        y,
        label=style_full["name"] + " mean",
        color=style_full["dark_color"],
    )
    ax.fill_betweenx(
        y,
        md_mean - 2 * md_sem,
        md_mean + 2 * md_sem,
        alpha=0.2,
        color=style_full["dark_color"],
    )

    ax.set_yticks(y_ticks)
    ax.set_yticklabels([])

    # mean and std
    # # median and IQR

    ax.plot(
        _x.median("cloud_id"),
        y,
        label=style_full["name"] + " median",
        color=style_full["color"],
        linestyle="--",
    )
    ax.fill_betweenx(
        y,
        _x.quantile(0.25, "cloud_id"),
        _x.quantile(0.75, "cloud_id"),
        alpha=0.2,
        color=style_full["light_color"],
    )

ax.set_yticks(y_ticks, y_ticks)
ax.legend(loc="upper right")
# ax.set_xlim(-10, 10)


ax.axvline(0, color="k", linestyle="--", alpha=1, zorder=10)
# ax.set_xscale('log')
ax.set_xlabel(label_from_attrs(x))
ax.set_ylabel(label_from_attrs(y, return_units=False, name_width=25))
# fig.suptitle("Evaporation profiles | Difference to EvapOnly")

fig.tight_layout()
save_figure(fig, fig_dir / f"profiles-{x.name}-{y.name}-LINEAR")

### Differences between microphysics 

In [ ]:
x_all = ds_normalized["evaporation_rate_energy"]
x_refernce = x_all.sel(microphysics="condensation")

x_sem_all = ds_normalized_sem["evaporation_rate_energy"]
x_sem_refernce = x_sem_all.sel(microphysics="condensation")

attrs = x_all.attrs.copy()

# f = (A - B)/B

# df/dA = 1/B
# df/dB = -A/B**2

# df = sqrt((df/dA * dA)**2 + (df/dB * dB)**2)
# df = sqrt((1/B * dA)**2 + (-A/B**2 * dB)**2)

A = x_all
B = x_refernce
dA = x_sem_all
dB = x_sem_refernce

f = A - B
df = (dA**2 + dB**2) ** 0.5

x = f
x_sem = df

x.attrs.update(
    long_name=f"{attrs['long_name']}\n difference to {microphysics_styles['condensation']['name']}",
    units=attrs["units"],
)

y = ds_normalized["normalized_gridbox_coord3"]


y_ticks = np.arange(0, 1.01, 0.25)

fig = plt.figure(figsize=(10, 5))
gs = fig.add_gridspec(nrows=3, ncols=6)

ax = fig.add_subplot(gs[:, :3])
ax0 = fig.add_subplot(gs[0, 3:])
ax1 = fig.add_subplot(gs[1, 3:], sharey=ax0)  # , sharex=ax0)
ax2 = fig.add_subplot(gs[2, 3:], sharey=ax0)  # , sharex=ax0)

plot_microphysics = [
    "collision_condensation",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]

axs = dict(
    zip(
        plot_microphysics,
        [ax0, ax1, ax2],
    )
)

for mp in plot_microphysics:

    _x = x.sel(microphysics=mp)
    _x_std = x_sem.sel(microphysics=mp)
    md_mean = _x.mean("cloud_id")
    md_std = propagate_error(_x, _x_std, dim="cloud_id")

    style_full = microphysics_styles[mp]
    style_full.update(name="")

    axs[mp].plot(
        _x.T,
        y.T,
        color=adjust_lightness(style_full["light_color"], 1.2),
        alpha=0.1,
        zorder=1,
    )
    axs[mp].plot(
        md_mean,
        y,
        label=style_full["name"],
        color=style_full["dark_color"],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])

    # mean and std

    ax.plot(
        md_mean,
        y,
        label=style_full["name"],
        color=style_full["dark_color"],
    )
    ax.fill_betweenx(
        y,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.1,
        color=style_full["color"],
    )

    # median and IQR

    ax.plot(
        _x.median("cloud_id"),
        y,
        # label=style_full['name'],
        color=style_full["color"],
        linestyle="--",
    )
    ax.fill_betweenx(
        y,
        _x.quantile(0.25, "cloud_id"),
        _x.quantile(0.75, "cloud_id"),
        alpha=0.1,
        color=style_full["light_color"],
    )


ax.plot(np.nan, np.nan, linestyle="--", color=adjust_lightness("k", 1.5), label="Median")
ax.plot(np.nan, np.nan, linestyle="-", color=adjust_lightness("k", 1.25), label="Mean")

ax.set_yticks(y_ticks, y_ticks)
ax.legend(loc="upper right")
ax.set_xlim(-10, 10)


for _ax in axs.values():
    _ax.axvline(0, color="k", linestyle="--", alpha=0.5)
    _ax.set_xlim(-100, 100)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)

fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# fig.suptitle("Evaporation profiles | Difference to EvapOnly")
fig.tight_layout()

add_subplotlabel(axs=np.array([ax] + list(axs.values())))

fig.tight_layout()

save_figure(fig, fig_dir / "evaporation_profiles_diff-normalized")

In [ ]:
x_all = ds_normalized["evaporation_rate_energy"]
x_refernce = x_all.sel(microphysics="condensation")

x_sem_all = ds_normalized_sem["evaporation_rate_energy"]
x_sem_refernce = x_sem_all.sel(microphysics="condensation")

attrs = x_all.attrs.copy()

# f = (A - B)/B

# df/dA = 1/B
# df/dB = -A/B**2

# df = sqrt((df/dA * dA)**2 + (df/dB * dB)**2)
# df = sqrt((1/B * dA)**2 + (-A/B**2 * dB)**2)

A = x_all
B = x_refernce
dA = x_sem_all
dB = x_sem_refernce

f = (A - B) / B
df = ((1 / B * dA) ** 2 + (-A / B**2 * dB) ** 2) ** 0.5

x = f * 100
x_sem = df * 100

x.attrs.update(
    long_name=f"{attrs['long_name']}\nrelative difference to {microphysics_styles['condensation']['name']}",
    units=r"\%",
)

y = ds_normalized["normalized_gridbox_coord3"]


y_ticks = np.arange(0, 1.01, 0.25)

fig = plt.figure(figsize=(10, 5))
gs = fig.add_gridspec(nrows=3, ncols=6)

ax = fig.add_subplot(gs[:, :3])
ax0 = fig.add_subplot(gs[0, 3:])
ax1 = fig.add_subplot(gs[1, 3:], sharey=ax0)  # , sharex=ax0)
ax2 = fig.add_subplot(gs[2, 3:], sharey=ax0)  # , sharex=ax0)

plot_microphysics = [
    "collision_condensation",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]

axs = dict(
    zip(
        plot_microphysics,
        [ax0, ax1, ax2],
    )
)

for mp in plot_microphysics:

    _x = x.sel(microphysics=mp)
    _x_std = x_sem.sel(microphysics=mp)
    md_mean = _x.mean("cloud_id")
    md_std = propagate_error(_x, _x_std, dim="cloud_id")

    style_full = microphysics_styles[mp]
    style_full.update(name="")

    axs[mp].plot(
        _x.T,
        y.T,
        color=adjust_lightness(style_full["light_color"], 1.2),
        alpha=0.1,
        zorder=1,
    )
    axs[mp].plot(
        md_mean,
        y,
        label=style_full["name"],
        color=style_full["dark_color"],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])

    # mean and std

    ax.plot(
        md_mean,
        y,
        label=style_full["name"],
        color=style_full["dark_color"],
    )
    ax.fill_betweenx(
        y,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.1,
        color=style_full["color"],
    )

    # median and IQR

    ax.plot(
        _x.median("cloud_id"),
        y,
        # label=style_full['name'],
        color=style_full["color"],
        linestyle="--",
    )
    ax.fill_betweenx(
        y,
        _x.quantile(0.25, "cloud_id"),
        _x.quantile(0.75, "cloud_id"),
        alpha=0.1,
        color=style_full["light_color"],
    )


ax.plot(np.nan, np.nan, linestyle="--", color=adjust_lightness("k", 1.5), label="Median")
ax.plot(np.nan, np.nan, linestyle="-", color=adjust_lightness("k", 1.25), label="Mean")

ax.set_yticks(y_ticks, y_ticks)
ax.legend(loc="upper right")
ax.set_xlim(-10, 10)


for _ax in axs.values():
    _ax.axvline(0, color="k", linestyle="--", alpha=0.5)
    _ax.set_xlim(-100, 100)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)

fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# fig.suptitle("Evaporation profiles | Difference to EvapOnly")
fig.tight_layout()

add_subplotlabel(axs=np.array([ax] + list(axs.values())))

fig.tight_layout()

save_figure(fig, fig_dir / "evaporation_profiles_diff-percentile-normalized")

In [ ]:
x_all = ds_normalized["evaporation_rate_energy"]
x_refernce = x_all.sel(microphysics="condensation")

x_sem_all = ds_normalized_sem["evaporation_rate_energy"]
x_sem_refernce = x_sem_all.sel(microphysics="condensation")

attrs = x_all.attrs.copy()

# f = (A - B)/B

# df/dA = 1/B
# df/dB = -A/B**2

# df = sqrt((df/dA * dA)**2 + (df/dB * dB)**2)
# df = sqrt((1/B * dA)**2 + (-A/B**2 * dB)**2)

A = x_all
B = x_refernce
dA = x_sem_all
dB = x_sem_refernce

f = (A - B) / B
df = ((1 / B * dA) ** 2 + (-A / B**2 * dB) ** 2) ** 0.5

x = f * 100
x_sem = df * 100

x.attrs.update(
    long_name=f"{attrs['long_name']}\nrelative difference to {microphysics_styles['condensation']['name']}",
    units=r"\%",
)

y = ds_normalized["normalized_gridbox_coord3"]


y_ticks = np.arange(0, 1.01, 0.25)

fig = plt.figure(figsize=(10, 5))
gs = fig.add_gridspec(nrows=3, ncols=6)

ax = fig.add_subplot(gs[:, :3])
ax0 = fig.add_subplot(gs[0, 3:])
ax1 = fig.add_subplot(gs[1, 3:], sharey=ax0)  # , sharex=ax0)
ax2 = fig.add_subplot(gs[2, 3:], sharey=ax0)  # , sharex=ax0)

plot_microphysics = [
    "collision_condensation",
    "coalbure_condensation_small",
    "coalbure_condensation_large",
]

axs = dict(
    zip(
        plot_microphysics,
        [ax0, ax1, ax2],
    )
)

for mp in plot_microphysics:

    _x = x.sel(microphysics=mp)
    _x_sem = x_sem.sel(microphysics=mp)
    md_mean = _x.mean("cloud_id")
    md_std = propagate_error(
        _x,
        _x_sem,
        dim="cloud_id",
    )

    style_full = microphysics_styles[mp]
    style_full.update(name="")

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])

    # mean and std

    ax.plot(
        md_mean,
        y,
        label=style_full["name"],
        color=style_full["dark_color"],
    )
    ax.fill_betweenx(
        y,
        md_mean - md_std,
        md_mean + md_std,
        alpha=0.1,
        color=style_full["color"],
    )

    # median and IQR

    ax.plot(
        _x.median("cloud_id"),
        y,
        # label=style_full['name'],
        color=style_full["color"],
        linestyle="--",
    )
    ax.fill_betweenx(
        y,
        _x.quantile(0.25, "cloud_id"),
        _x.quantile(0.75, "cloud_id"),
        alpha=0.1,
        color=style_full["light_color"],
    )

    axs[mp].set_yticks(y_ticks)
    axs[mp].set_yticklabels([])

    axs[mp].plot(
        md_mean,
        y,
        label=style_full["name"],
        color=style_full["dark_color"],
        linestyle="-",
    )
    # median and IQR

    axs[mp].plot(
        _x.median("cloud_id"),
        y,
        # label=style_full['name'],
        color=style_full["color"],
        linestyle="--",
        label="Median",
    )

    axs[mp].fill_betweenx(
        y,
        _x.quantile(0.1, "cloud_id"),
        _x.quantile(0.9, "cloud_id"),
        alpha=0.1,
        color=style_full["light_color"],
        label="10-90%",
    )
    axs[mp].fill_betweenx(
        y,
        _x.quantile(0.25, "cloud_id"),
        _x.quantile(0.75, "cloud_id"),
        alpha=0.1,
        color=style_full["light_color"],
        label="25-75%",
    )
    axs[mp].fill_betweenx(
        y,
        _x.quantile(0.33, "cloud_id"),
        _x.quantile(0.66, "cloud_id"),
        alpha=0.1,
        color=style_full["light_color"],
        label="33-66%",
    )


ax.plot(np.nan, np.nan, linestyle="--", color=adjust_lightness("k", 1.5), label="Median")
ax.plot(np.nan, np.nan, linestyle="-", color=adjust_lightness("k", 1.25), label="Mean")

ax.set_yticks(y_ticks, y_ticks)
ax.legend(loc="upper right")
ax.set_xlim(-10, 10)

# ax0.set_xlim(-50, 100)

for _ax in axs.values():
    _ax.axvline(0, color="k", linestyle="--", alpha=0.5)
    _ax.set_xlim(-15, 15)
ax.axvline(0, color="k", linestyle="--", alpha=0.5)

fig.supxlabel(label_from_attrs(x))
fig.supylabel(label_from_attrs(y))
# fig.suptitle("Evaporation profiles | Difference to EvapOnly")
fig.tight_layout()

add_subplotlabel(axs=np.array([ax] + list(axs.values())))


fig.tight_layout()
save_figure(fig, fig_dir / "evaporation_profiles_diff-percentile-normalized-distribution")

In [ ]:
x_all = ds_normalized["evaporation_rate_energy"]
x_refernce = x_all.sel(microphysics="condensation")

x_sem_all = ds_normalized_sem["evaporation_rate_energy"]
x_sem_refernce = x_sem_all.sel(microphysics="condensation")

attrs = x_all.attrs.copy()

# f = (A - B)/B

# df/dA = 1/B
# df/dB = -A/B**2

# df = sqrt((df/dA * dA)**2 + (df/dB * dB)**2)
# df = sqrt((1/B * dA)**2 + (-A/B**2 * dB)**2)

A = x_all
B = x_refernce
dA = x_sem_all
dB = x_sem_refernce

f = (A - B) / B
df = ((1 / B * dA) ** 2 + (-A / B**2 * dB) ** 2) ** 0.5

x = f * 100
x_sem = df * 100

x.attrs.update(
    long_name=f"{attrs['long_name']}\nrelative difference to {microphysics_styles['condensation']['name']}",
    units=r"\%",
)

y = ds_normalized["normalized_gridbox_coord3"]


y_ticks = np.arange(0, 1.01, 0.25)

heights = np.linspace(0.98, 0, 5)
heights = np.array([1, 0.98, 0.75, 0.5, 0.25, 0])


fig, axs = plt.subplots(nrows=len(heights), figsize=(5, 8))

plot_microphysics = [
    "collision_condensation",
    "coalbure_condensation_large",
    "coalbure_condensation_small",
]

for h, ax in zip(heights, axs):
    x_sel = x.sel(normalized_gridbox_coord3=h, method="nearest")
    # x_sel_sel = x_sel.where(np.abs(x_sel) < 100, drop = True)
    for mp in plot_microphysics:
        style = microphysics_styles.get_style(mp, colortype="light")
        style.update(linestyle="-")
        style.pop("marker")

        sns.kdeplot(
            data=x_sel.sel(microphysics=mp),
            ax=ax,
            fill=True,
            alpha=0.1,
            linewidth=1,
            bw_adjust=0.5,
            # legend=plot_microphysics,
            **style,
        )

        ax.axvline(x_sel.sel(microphysics=mp).mean(), color=microphysics_styles[mp]["dark_color"])
        ax.axvline(
            x_sel.sel(microphysics=mp).median(), color=microphysics_styles[mp]["color"], linestyle="--"
        )

    ax.set_ylabel(f"{h}\n\nKDE")
    ax.axvline(0, color="k", linestyle=":", alpha=0.5)

fig.supylabel(label_from_attrs(y), x=-0.05)

for ax in axs:
    ax.set_xlim(-10, 10)
    ax.spines["left"].set_visible(False)
    ax.spines["bottom"].set_visible(False)

    ax.plot(np.nan, np.nan, linestyle="--", color=adjust_lightness("k", 1.5), label="Median")
    ax.plot(np.nan, np.nan, linestyle="-", color=adjust_lightness("k", 1.25), label="Mean")
    ax.legend()

for ax in axs[:-1]:
    ax.set_xticks([])


fig.suptitle("Distribution of relative evaporation difference\nover different vertical levels")
fig.supxlabel(label_from_attrs(x))
fig.tight_layout()

save_figure(fig=fig, filepath=fig_dir / "distributiions-evaporation_profiles_diff-percentile")